In [1]:
# turn autoreload on for Jupyter notebook 
%load_ext autoreload
%autoreload 2


# Voting Scripts

This notebook contains implementations of voting methods and class to reason about voting situations. 

The code here is written to the files in the voting directory, so it can be easily imported into other notebooks. 


In [2]:

# fix some candidates names

A = "A"
B = "B"
C = "C"
D = "D"
E = "E"


## Rankings

Suppose that $X$ is a finite set of candidates, or alternatives.   A **ranking** on $X$ is a relation $R\subseteq X\times X$ that is irreflexive (for all $A\in X$, it is not the case that $A\mathrel{R} A$), transitive  (for all $A, B, C\in X$, if $A\mathrel{R} B$ and $B\mathrel{R}C$, then $A\mathrel{R} C$),  and complete (for all $A, B\in X$ if $A\ne B$, then $A\mathrel{R} B$ or $B\mathrel{R} A$).  I.e., a ranking is a *linear order* on $X$.  




In [3]:
#%%writefile ./voting/ranking.py

class Ranking():
    '''Ranking: a ranking of a set of candidates is a linear order of the candidates'''
    def __init__(self, cand_list):
        # cand_list is a list of candidates 
        
        # make sure that there are no duplicates
        assert len(list(set(cand_list))) == len(cand_list), "Cannot create a ranking when there are duplicates is the candidate ordering {}".format(cand_list)
        
        # a list of candidate records: (rank, candidate_name)
        self._ranking = list(enumerate(cand_list))
    
    @property
    def num_candidates(self):
        return len(self._ranking)
    
    @property
    def candidates(self):
        return set([cr[1] for cr in self._ranking])
    
    def _get_cr_by_cand(self, cand):      
        # get the candidate rank given the name of the candidate
        
        assert cand in self.candidates
        
        return [cr for cr in self._ranking if cr[1] == cand][0]

    def _get_cr_by_rank(self, rank):
        # get the candidate record by the rank
        
        assert rank >= 1 and rank <= len(list(self.candidates)) + 1
        
        return [cr for cr in self._ranking if cr[0] == rank - 1][0]

    def rank(self,cand):
        # return the rank of a candidate
        
        return self._get_cr_by_cand(cand)[0] + 1
    
    def cand(self,rank):
        # return the candidate positioned at rank
        return self._get_cr_by_rank(rank)[1]
        
    def __str__(self):
        # return the list of candidates as a string
        
        return " ".join([cr[1] for cr in self._ranking])
    
    def as_tuple(self):
        # return the ranking as a tuple 
        return tuple([cr[1] for cr in self._ranking])
    
    def __eq__(self, other): 
        if isinstance(other, self.__class__):
            return list(self._ranking) == list(other._ranking)
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)        
        


In [4]:
        
r = Ranking((B, C, A))
print "The ranking is {}".format(r)
print "There number of candidates in the ranking is {}".format(r.num_candidates)
print "The candidate ranked 1st is {}".format(r.cand(1))
print "The candidate ranked 2nd is {}".format(r.cand(2))
print "The candidate ranked 3rd is {}".format(r.cand(3))
print "Candidate {} is ranked {}".format(A,r.rank(A))
print "Candidate {} is ranked {}".format(B,r.rank(B))
print "Candidate {} is ranked {}".format(C,r.rank(C))
print "{} is equal to {}: {}".format(r, Ranking((B, C, A)), r == Ranking((B, C, A)))
print "{} is not equal to {}: {}".format(r, Ranking((A, B, C)), r != Ranking((A, B, C)))
print "The ranking is (as a string): {}".format(str(r))
print "The ranking is (as a tuple): {}".format(r.as_tuple())

The ranking is B C A
There number of candidates in the ranking is 3
The candidate ranked 1st is B
The candidate ranked 2nd is C
The candidate ranked 3rd is A
Candidate A is ranked 3
Candidate B is ranked 1
Candidate C is ranked 2
B C A is equal to B C A: True
B C A is not equal to A B C: True
The ranking is (as a string): B C A
The ranking is (as a tuple): ('B', 'C', 'A')


## Voters

Each voter is associated with a ranking.  In addition to a ranking over the candidates $R$, voters are associated with 

* weak preference over the set of candidates (reflexive closure of the voter's ranking)
* lifting of $R$ to relation on sets of candidates: AAdom, weak_dom, (strong_dom), (weak) optimist, (weak) pessimist
* dominance relations comparing sets of sets of candidates: sure_dominance, safe_dominance, expected_dominance
* probabilistic notions: probability generated by sets of sets of candidates, stochastic dominance

In [5]:
#%%writefile ./voting/voter.py

class Voter():
    def __init__(self, name, ranking = None):
        
        self.name = name
        self.ranking = ranking
        
    def candidate_list(self):
        return list(self.ranking.candidates)
    
    # voter preferences 
    def prefers(self, c1, c2):
        # return True if c1 is ranked above c2
        
        # check that candidates are in the rankings
        assert c1 in self.ranking.candidates, "Error: {} not in the voters ranking {}".format(c1, self.ranking)
        assert c2 in self.ranking.candidates, "Error: {} not in the voters ranking {}".format(c2, self.ranking)
        
        # lower rank number means the candidate is ranked higher
        return self.ranking.rank(c1) < self.ranking.rank(c2)

    def wprefers(self, c1, c2):         
        # return True if c1 is ranked above c2 or c1 == c2
        
        # check that candidates are in the rankings
        assert c1 in self.ranking.candidates, "Error: {} not in the voters ranking {}".format(c1, self.ranking)
        assert c2 in self.ranking.candidates, "Error: {} not in the voters ranking {}".format(c2, self.ranking)
        
        return c1 == c2 or self.ranking.rank(c1) < self.ranking.rank(c2)

    def rank_candidate(self, rank):
        # return the candidate at position rank
        
        return self.ranking.cand(rank)
    
    def rank(self, candidate):
        # return the ranking of candidate
        
        return self.ranking.rank(candidate)
    
    def first(self, cs = None):
        # return the first ranked candidate from a list of candidates cs (or all if cs is None)
        if cs is None:
            first = self.rank_candidate(1)
        else:
            first = min(cs, key = lambda c: self.rank(c))
        return first

    def last(self, cs = None):
        # return the last ranked candidate from a list of candidates cs (or all if cs is None)
        if cs is None:
            last = self.rank_candidate(self.ranking.num_candidates)
        else:
            last = max(cs, key = lambda c: self.rank(c))
        return last

    # set preferences
    def AAdom(self, c1s, c2s):         
        # return True if every candidate in c1s is weakly preferred to every  candidate in c2s
        
        # check if all candidates are ranked
        assert set(c1s).union(set(c2s)).issubset(self.ranking.candidates), "Error: candidates in the sets {} and {} are not ranked".format(c1s, c2s)
        return all([all([self.wprefers(c1, c2) for c2 in c2s]) for c1 in c1s])
    
    def strong_dom(self, c1s, c2s):         
        # return True if AAdom(c1s, c2s) and there is a candidate in c1s that is strictly preferred to every  candidate in c2s
        
        # check if all candidates are ranked
        assert set(c1s).union(set(c2s)).issubset(self.ranking.candidates), "Error: candidates in the sets {} and {} are not ranked".format(c1s, c2s)
        
        return self.AAdom(c1s, c2s) and any([all([self.prefers(c1, c2) for c2 in c2s]) for c1 in c1s])

    def weak_dom(self, c1s, c2s):         
        # return True if AAdom(c1s, c2s) and there is a candidate in c1s that is strictly preferred to every  candidate in c2s
        
        # check if all candidates are ranked
        assert set(c1s).union(set(c2s)).issubset(self.ranking.candidates), "Error: candidates in the sets {} and {} are not ranked".format(c1s, c2s)
        
        return self.AAdom(c1s, c2s) and any([any([self.prefers(c1, c2) for c2 in c2s]) for c1 in c1s])

    def optimist_prefers(self, c1s, c2s):
        # return True if the best candidate from c1s is strictly  preferred to the best candidate from c2s
        
        # check if all candidates are ranked
        assert set(c1s).union(set(c2s)).issubset(self.ranking.candidates), "Error: candidates in the sets {} and {} are not ranked".format(c1s, c2s)
        
        return self.prefers(self.first(c1s),self.first(c2s))

    def weak_optimist_prefers(self, c1s, c2s):
        # return True if the best candidate from c1s is weakly preferred to the best candidate from c2s
        
        # check if all candidates are ranked
        assert set(c1s).union(set(c2s)).issubset(self.ranking.candidates), "Error: candidates in the sets {} and {} are not ranked".format(c1s, c2s)
        
        return self.wprefers(self.first(c1s),self.first(c2s))

    def pessimist_prefers(self, c1s, c2s):
        # return True if the worst candidate from c1s is strictly  preferred to the worst candidate from c2s
        
        # check if all candidates are ranked
        assert set(c1s).union(set(c2s)).issubset(self.ranking.candidates), "Error: candidates in the sets {} and {} are not ranked".format(c1s, c2s)
        
        return self.prefers(self.last(c1s),self.last(c2s))

    def weak_pessimist_prefers(self, c1s, c2s):
        # return True if the worst candidate from c1s is weakly preferred to the worst candidate from c2s
        
        # check if all candidates are ranked
        assert set(c1s).union(set(c2s)).issubset(self.ranking.candidates), "Error: candidates in the sets {} and {} are not ranked".format(c1s, c2s)
        
        return self.wprefers(self.last(c1s),self.last(c2s))

    # dominance notions comparing collections of winning sets
    def sure_dominance(self, winning_set1, winning_set2, dom):
        
        # return True if winning_set1 surely dominantes winning_set2 using dom as the dominance notion
        assert len(winning_set1) == len(winning_set2), "Can only apply sure dominance to winning sets of the same size: {}, {}".format(winning_set1, winning_set2)
        
        return all([dom(ws[0], ws[1]) for ws in zip(winning_set1, winning_set2)])

    def safe_dominance(self, winning_set1, winning_set2, wdom, dom):
        
        # return True if winning_set1 safely dominantes winning_set2 using
        # wdom as the weak dominance notion and dom as the dominance notion
        assert len(winning_set1) == len(winning_set2), "Can only apply apply dominance to winning sets of the same size: {}, {}".format(winning_set1, winning_set2)
        
        weak_pref = all([wdom(ws[0], ws[1]) for ws in zip(winning_set1, winning_set2)])
        strict_pref = any([dom(ws[0], ws[1]) for ws in zip(winning_set1, winning_set2)])

        return weak_pref and strict_pref

    def expected_dominance(self, winning_set1, winning_set2, dom):
        
        # return True if winning_set1 expected dominantes winning_set2 using
        # dom as the dominance notion
        assert len(winning_set1) == len(winning_set2), "Can only apply expected dominance to winning sets of the same size: {}, {}".format(winning_set1, winning_set2)
        
        better_outcomes = [1 for ws in zip(winning_set1, winning_set2) if dom(ws[0], ws[1])]
        worse_outcomes = [1 for ws in zip(winning_set1, winning_set2) if dom(ws[1], ws[0])]

        return len(better_outcomes) > len(worse_outcomes)

    #ALTERNATIVE NOTION
    #def expected_dominance(self, winning_set1, winning_set2, dom):
        
        # return True if winning_set1 expected dominantes winning_set2 using
        # dom as the dominance notion
    #    assert len(winning_set1) == len(winning_set2), "Can only apply expected dominance to winning sets of the same size: {}, {}".format(winning_set1, winning_set2)
        
    #    better_outcomes = [1 for ws in zip(winning_set1, winning_set2) if dom(ws[0], ws[1])]
    #    not_better_outcomes = [1 for ws in zip(winning_set1, winning_set2) if not dom(ws[0], ws[1])]

    #    return len(better_outcomes) > len(not_better_outcomes)

    
    # stochastic dominance/probabilities of candidates givine winning sets
    def probability(self, winning_sets, candidates):
        
        # return a dictionary associating the probability that a candidate will win 
        num_methods = len(winning_sets)
        return {c: float(1)/float(num_methods) * sum([float(1)/len(ws) for ws in winning_sets if c in ws]) for c in candidates}


    def prob(self, cand, winning_sets, candidates):
        # return a the probability of the candidate chosen as the winner
        
        assert cand in candidates, "{} must be in the set {}".format(cand, candidates)
        return self.probability(winning_sets, candidates)[cand]

        
    def weak_stochastic_dominance(self, winning_set1, winning_set2, candidates):
        
        # return True if winning_set1 weak stochastic dominantes winning_set2 
        
        assert len(winning_set1) == len(winning_set2), "Can only apply weak stochastic dominance to winning sets of the same size: {}, {}".format(winning_set1, winning_set2)
        
        num_candidates = len(candidates)
        ranks = range(num_candidates, 0, -1)
        
        ws1_probs = self.probability(winning_set1, candidates)
        ws2_probs = self.probability(winning_set2, candidates)

        candidate_sets = [[voter.rank_candidate(r)  for r in ranks[_:len(ranks)+1]] for _ in range(0,len(ranks))]

        candidate_probs_1 =  [sum([ws1_probs[c] for c in _cands]) for _cands in candidate_sets]
        candidate_probs_2 =  [sum([ws2_probs[c] for c in _cands]) for _cands in candidate_sets]

        return all([p[0] >= p[1] for p in zip(candidate_probs_1, candidate_probs_2)])    
    
    def stochastic_dominance(self, winning_set1, winning_set2, candidates):
        
        # return True if winning_set1  stochastic dominantes winning_set2 
        
        assert len(winning_set1) == len(winning_set2), "Can only apply stochastic dominance to winning sets of the same size: {}, {}".format(winning_set1, winning_set2)
        
        num_candidates = len(candidates)
        ranks = range(num_candidates, 0, -1)
        
        ws1_probs = self.probability(winning_set1, candidates)
        ws2_probs = self.probability(winning_set2, candidates)

        candidate_sets = [[self.rank_candidate(r)  for r in ranks[_:len(ranks)+1]] for _ in range(0,len(ranks))]

        candidate_probs_1 =  [sum([ws1_probs[c] for c in _cands]) for _cands in candidate_sets]
        candidate_probs_2 =  [sum([ws2_probs[c] for c in _cands]) for _cands in candidate_sets]

        return all([p[0] > p[1] for p in zip(candidate_probs_1, candidate_probs_2)]) 



In [6]:
#%run voter

v=Voter("Ann",ranking=Ranking((B,C,A)))

print "{}'s ranking is {}".format(v.name, v.ranking)
print "{} prefers {} over {} is {}".format(v.name, A, B, v.prefers(A,B))
print "{} prefers {} over {} is {}".format(v.name, B, A, v.prefers(B,A))
print "{} prefers {} over {} is {}".format(v.name, A, A, v.prefers(A,A))
print "{} weakly prefers {} over {} is {}".format(v.name, A, B, v.wprefers(A,B))
print "{} weakly prefers {} over {} is {}".format(v.name, B, A, v.wprefers(B,A))
print "{} weakly prefers {} over {} is {}".format(v.name, A, A, v.wprefers(A,A))

print "{} is ranked 1st".format(v.rank_candidate(1))
print "{} is ranked 2nd".format(v.rank_candidate(2))
print "{} is ranked 3rd".format(v.rank_candidate(3))

print "{} is ranked {}".format(A, v.rank(A))
print "{} is ranked {}".format(B, v.rank(B))
print "{} is ranked {}".format(C, v.rank(C))



print "{} is ranked first".format(v.first())
print "The top ranked candidate from {} is {}".format([A,C], v.first([A,C]))
print "The top ranked candidate from {} is {}".format([A,B], v.first([A,B]))
print "The top ranked candidate from {} is {}".format([B,C], v.first([B,C]))
print "The top ranked candidate from {} is {}".format([A,B,C], v.first([A,B,C]))

print "{} is ranked last".format(v.last())
print "The bottom ranked candidate from {} is {}".format([A,C], v.last([A,C]))
print "The bottom ranked candidate from {} is {}".format([A,B], v.last([A,B]))
print "The bottom ranked candidate from {} is {}".format([B,C], v.last([B,C]))
print "The bottom ranked candidate from {} is {}".format([A,B,C], v.last([A,B,C]))

print "\n-----------------\n"
print "AAdominance"
print "{} AA-dominates {} is {}".format([A,B], [B,C],v.AAdom([A,B], [B,C]))
print "{} AA-dominates {} is {}".format([B,C], [A,B],v.AAdom([B,C], [A,B]))
print "{} AA-dominates {} is {}".format([B,C], [A,C],v.AAdom([B,C], [A,C]))
print "{} AA-dominates {} is {}".format([B], [A,C],v.AAdom([B], [A,C]))

print "\nStrong Dominance"
print "{} strong-dominates {} is {}".format([A,B], [B,C],v.strong_dom([A,B], [B,C]))
print "{} strong-dominates {} is {}".format([B,C], [A,B],v.strong_dom([B,C], [A,B]))
print "{} strong-dominates {} is {}".format([B,C], [A,C],v.strong_dom([B,C], [A,C]))
print "{} strong-dominates {} is {}".format([B], [A,C],v.strong_dom([B], [A,C]))
print "{} strong-dominates {} is {}".format([B], [A,B],v.strong_dom([B], [A, B]))

print "\nWeak Doinance"
print "{} weak-dominates {} is {}".format([A,B], [B,C],v.weak_dom([A,B], [B,C]))
print "{} weak-dominates {} is {}".format([B,C], [A,B],v.weak_dom([B,C], [A,B]))
print "{} weak-dominates {} is {}".format([B,C], [A,C],v.weak_dom([B,C], [A,C]))
print "{} weak-dominates {} is {}".format([B], [A,C],v.weak_dom([B], [A,C]))
print "{} weak-dominates {} is {}".format([B], [A,B],v.weak_dom([B], [A,B]))

print "\nOptimist Prefers"
print "{} optimist prefers {} is {}".format([A,B], [B,C],v.optimist_prefers([A,B], [B,C]))
print "{} optimist prefers {} is {}".format([B,C], [A,B],v.optimist_prefers([B,C], [A,B]))
print "{} optimist prefers {} is {}".format([B,C], [A,C],v.optimist_prefers([B,C], [A,C]))
print "{} optimist prefers {} is {}".format([B], [A,C],v.optimist_prefers([B], [A,C]))
print "{} optimist prefers {} is {}".format([B], [A,B],v.optimist_prefers([B], [A,B]))

print "\nWeak Optimist Prefers"
print "{} weak optimist prefers {} is {}".format([A,B], [B,C],v.weak_optimist_prefers([A,B], [B,C]))
print "{} weak optimist prefers {} is {}".format([B,C], [A,B],v.weak_optimist_prefers([B,C], [A,B]))
print "{} weak optimist prefers {} is {}".format([B,C], [A,C],v.weak_optimist_prefers([B,C], [A,C]))
print "{} weak optimist prefers {} is {}".format([B], [A,C],v.weak_optimist_prefers([B], [A,C]))
print "{} weak optimist prefers {} is {}".format([B], [A,B],v.weak_optimist_prefers([B], [A,B]))

print "\nPessimist Prefers"
print "{} pessimist prefers {} is {}".format([A,B], [B,C],v.pessimist_prefers([A,B], [B,C]))
print "{} pessimist prefers {} is {}".format([B,C], [A,B],v.pessimist_prefers([B,C], [A,B]))
print "{} pessimist prefers {} is {}".format([B,C], [A,C],v.pessimist_prefers([B,C], [A,C]))
print "{} pessimist prefers {} is {}".format([B], [A,C],v.pessimist_prefers([B], [A,C]))
print "{} pessimist prefers {} is {}".format([B], [A,B],v.pessimist_prefers([B], [A,B]))
print "{} pessimist prefers {} is {}".format([A, B], [B],v.pessimist_prefers([A,B], [B]))
print "{} pessimist prefers {} is {}".format([A, B], [A, C],v.pessimist_prefers([A, B], [A, C]))

print "\nWeak Pessimist Prefers"
print "{} weak pessimist prefers {} is {}".format([A,B], [B,C],v.weak_pessimist_prefers([A,B], [B,C]))
print "{} weak pessimist prefers {} is {}".format([B,C], [A,B],v.weak_pessimist_prefers([B,C], [A,B]))
print "{} weak pessimist prefers {} is {}".format([B,C], [A,C],v.weak_pessimist_prefers([B,C], [A,C]))
print "{} weak pessimist prefers {} is {}".format([B], [A,C],v.weak_pessimist_prefers([B], [A,C]))
print "{} weak pessimist prefers {} is {}".format([B], [A,B],v.weak_pessimist_prefers([B], [A,B]))
print "{} weak pessimist prefers {} is {}".format([A, B], [B],v.weak_pessimist_prefers([A,B], [B]))
print "{} weak pessimist prefers {} is {}".format([A, B], [A, C],v.weak_pessimist_prefers([A, B], [A, C]))

print "\nSure weak dominance"
print "{} sure weak dominates {} is {}".format([[A], [A,B]], 
                                               [[A], [C, B]], 
                                               v.sure_dominance([[A], [A,B]], [[A], [C, B]], v.weak_dom))
print "{} sure weak dominates {} is {}".format([[B], [C]], 
                                               [[C], [A]], 
                                               v.sure_dominance([[B], [C]], [[C], [A]], v.weak_dom))
print "{} sure weak dominates {} is {}".format([[B], [C]], 
                                               [[C], [C]], 
                                               v.sure_dominance([[B], [C]], [[C], [C]], v.weak_dom))
print "\nSafe weak dominance"
print "{} safe weak dominates {} is {}".format([[A], [A,B]], 
                                               [[A], [C, B]], 
                                               v.safe_dominance([[A], [A,B]], [[A], [C, B]], v.AAdom, v.weak_dom))
print "{} safe weak dominates {} is {}".format([[B], [C]], 
                                               [[C], [A]], 
                                               v.safe_dominance([[B], [C]], [[C], [A]], v.AAdom, v.weak_dom))
print "{} safe weak dominates {} is {}".format([[B,C], [C]], 
                                               [[C], [C]], 
                                               v.safe_dominance([[B], [C]], [[C], [C]], v.AAdom, v.weak_dom))
print "{} safe weak dominates {} is {}".format([[B,C], [C], [A]], 
                                               [[C], [C], [C]], 
                                               v.safe_dominance([[B,C], [C], [A]], [[C], [C], [C]], v.AAdom, v.weak_dom))
print "{} safe weak dominates {} is {}".format([[B,C], [B], [A]], 
                                               [[C], [C], [C]], 
                                               v.safe_dominance([[B,C], [B], [A]], [[C], [C], [C]], v.AAdom, v.weak_dom))

print "\nExpected weak dominance"
print "{} expected weak dominates {} is {}".format([[A], [A,B]], 
                                               [[A], [C, B]], 
                                               v.expected_dominance([[A], [A,B]], [[A], [C, B]],   v.weak_dom))
print "{} expected weak dominates {} is {}".format([[B], [C]], 
                                               [[C], [A]], 
                                               v.expected_dominance([[B], [C]], [[C], [A]],   v.weak_dom))
print "{} expected weak dominates {} is {}".format([[B,C], [C]], 
                                               [[C], [C]], 
                                               v.expected_dominance([[B], [C]], [[C], [C]],   v.weak_dom))
print "{} expected weak dominates {} is {}".format([[B,C], [C], [A]], 
                                               [[C], [C], [C]], 
                                               v.expected_dominance([[B,C], [C], [A]], [[C], [C], [C]], v.weak_dom))
print "{} expected weak dominates {} is {}".format([[B,C], [B], [A]], 
                                               [[C], [C], [C]], 
                                               v.expected_dominance([[B,C], [B], [A]], [[C], [C], [C]], v.weak_dom))
print "{} expected weak dominates {} is {}".format([[B,C], [C], [C]], 
                                               [[C], [C], [A]], 
                                               v.expected_dominance([[B,C], [C], [C]], [[C], [C], [A]], v.weak_dom))


print "\nProbability"
print "The probabilities for {} is {}".format([[A], [A,B]],                                               
                                              v.probability([[A], [A,B]], [A,B,C]))
print "The prob of {} for {} is {}".format(A, [[A], [A,B]],                                               
                                            v.prob(A, [[A], [A,B]], [A,B,C]))
print "The prob of {} for {} is {}".format(B, [[A], [A,B]],                                               
                                            v.prob(B, [[A], [A,B]], [A,B,C]))
print "The prob of {} for {} is {}".format(C, [[A], [A,B]],                                               
                                            v.prob(C, [[A], [A,B]], [A,B,C]))
print "The probabilities for {} is {}".format([[A,C], [A,B]],                                               
                                              v.probability([[A,C], [A,B]], [A,B,C]))

print "{} stochastic dominates {} is {}".format([[B,C], [C], [C]],
                                                [[C], [C], [A]], 
                                                v.stochastic_dominance([[B,C], [C], [C]], [[C], [C], [A]], [A, B, C]))
print "{} stochastic dominates {} is {}".format([[B], [B], [C]],
                                                [[C], [A], [A]], 
                                                v.stochastic_dominance([[B], [B], [C]], [[C], [A], [A]], [A, B, C]))


Ann's ranking is B C A
Ann prefers A over B is False
Ann prefers B over A is True
Ann prefers A over A is False
Ann weakly prefers A over B is False
Ann weakly prefers B over A is True
Ann weakly prefers A over A is True
B is ranked 1st
C is ranked 2nd
A is ranked 3rd
A is ranked 3
B is ranked 1
C is ranked 2
B is ranked first
The top ranked candidate from ['A', 'C'] is C
The top ranked candidate from ['A', 'B'] is B
The top ranked candidate from ['B', 'C'] is B
The top ranked candidate from ['A', 'B', 'C'] is B
A is ranked last
The bottom ranked candidate from ['A', 'C'] is A
The bottom ranked candidate from ['A', 'B'] is A
The bottom ranked candidate from ['B', 'C'] is C
The bottom ranked candidate from ['A', 'B', 'C'] is A

-----------------

AAdominance
['A', 'B'] AA-dominates ['B', 'C'] is False
['B', 'C'] AA-dominates ['A', 'B'] is False
['B', 'C'] AA-dominates ['A', 'C'] is True
['B'] AA-dominates ['A', 'C'] is True

Strong Dominance
['A', 'B'] strong-dominates ['B', 'C'] is Fal

## Profile

Formally, a **profile** is an element of $L(X)^V$, where $V$ is the set of voters.  

A **pointed profile** is a profile with a designated voter

In [7]:
#%%writefile ./voting/profile.py

import itertools
from prettytable import *
from math import ceil

from voting.voter import Voter
from voting.ranking import Ranking

###
#Helper functions
###

def generate_linear_orderings(candidates): 
    # returns all linear orderings for a set of candidates
    
    return list(itertools.permutations(candidates))

def create_subsets(s, n): 
    # return all subsets of s of size n
    
    return list(itertools.combinations(s, n)) 


# Profiles
class Profile(object):
    
    def __init__(self, voter_list):
        
        assert len(voter_list) > 0, "Error: can only  create a profile with more than 0 voters"
        self._profile = voter_list
        self.voter_names = sorted([v.name for v in self._profile])
    
    @property
    def voters(self):
        return self._profile
    
    @property 
    def candidates(self):
        return self._profile[0].candidate_list()
    
    @property
    def num_candidates(self):
        return len(self.candidates)
    
    @property
    def all_rankings(self):
        return generate_linear_orderings(self.candidates)
    
    def get_voter(self, voter_name):
        
        # return the voter given the voter_name
        
        assert voter_name in self.voter_names, "Error: {} voter name is not in the profile: {}".format(voter_name, self.voter_names)
        
        voter = [v for v in self._profile if v.name == voter_name][0]
            
        return voter
    
    def support(self, c1, c2):

        # return the number of voters that rank c1 over c2
        _support  = 0
        for v in self._profile:
            if v.prefers(c1, c2):
                _support +=1
        return _support
    
    def create_new_profile(self, voter, new_ordering):
        # given a voter and a new ordering, create a new profile exactly like the existing one
        return Profile([v if v.name != voter.name else Voter(voter.name, ranking = Ranking(new_ordering)) 
                        for v in self._profile])
    
    def remove_candidates(self, cands):
        # create a new profile with the candidates removed
        
        if type(cands) == str: 
            cands = [cands]
            
        new_voter_list = [Voter(v.name,ranking=Ranking([_ for _ in str(v.ranking).split(' ') if _ not in cands])) for v in self.voters]
        return Profile(new_voter_list)
    
    def is_empty(self):
        # return true if some ranking is empty
        return any([len(v.ranking.candidates) == 0 for v in self._profile])
    
    def tally(self):

        # generate a tally  for all the candidates.   This is a dictionary associating with each voter, the 
        # margin of victory for that candidate over the other candidates. 

        return {c1: {c2: self.support(c1, c2) - self.support(c2, c1) for c2 in self.candidates if c2 != c1} 
                for c1 in self.candidates}

    def majority_prefers(self, c1, c2): 
        # return True if more voters rank c1 over c2 than c2 over c1

        return (self.support(c1, c2) - self.support(c2, c1)) > 0

    def num_rank(self, cand, level = 1):
        # returns the number of voters that rank cand at level
        
        return sum(v.rank_candidate(level) == cand for v in self._profile)

    def plurality_tally(self):
        #  return  the pluarlity tally
        
        return {c: self.num_rank(c, level=1) for c in self.candidates}

    def borda_scores(self):
        # return the Borda scores for each candidate
        
        borda_scores = {c:0 for c in self.candidates}
        num_candidates = len(self.candidates)
        for v in self._profile:
            borda_scores.update({c:((num_candidates - v.rank(c)) + borda_scores[c]) for c in self.candidates})
        return borda_scores
    
    def strict_maj_size(self):
        
        # return the size of  > 50% of the voters
        return len(self.voters)/2 + 1 if len(self.voters) % 2 == 0 else int(ceil(float(len(self.voters))/2))

    def display(self):
        tbl = PrettyTable()
        for v in self._profile:
            tbl.add_column(v.name, str(v.ranking).split(' '))
        print tbl

    def get_profile_string(self):
        tbl = PrettyTable()
        for v in self._profile:
            tbl.add_column(v.name, str(v.ranking).split(' '))
        return str(tbl)

    def as_dict(self): 
        return {v.name: v.ranking.as_tuple() for v in self._profile}
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return all([vs[0].name == vs[1].name and vs[0].ranking == vs[1].ranking for vs in zip(self._profile, 
                                                                                                   other._profile)])
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)
    
# a pointed profile extends the Profile class
class PointedProfile(Profile):
    
    def __init__(self, voter_list, voter_name):
        super(PointedProfile, self).__init__(voter_list)
        
        assert voter_name in [v.name for v in voter_list], "Error: {} voter name is not in the list of voters".format(voter_name)
        self._voter_pos = [v.name for v in voter_list].index(voter_name)
        self._voter_name = voter_name
        
    @property
    def voter(self):
        return self._profile[self._voter_pos]
        
    def create_new_profile(self, new_ranking):
        # given a voter and a new ordering, create a new profile exactly like the existing one
        return PointedProfile([v if v.name != self._voter_name else Voter(self._voter_name,
                                                                          ranking = Ranking(new_ranking)) 
                               for v in self._profile], self._voter_name)    
###
#Helper functions
###

def create_profile(voter_names, rankings):
    # given voter names and a list of rankings, return a profile
    
    assert len(voter_names) == len(rankings), "Error: the number of voters doesn't match the number of rankings"
    return Profile([Voter(vr[0], ranking = Ranking(vr[1])) for vr in zip(voter_names, rankings)])

def create_pointed_profile(voter_names, rankings, voter):
    # given voter names and a list of rankings, return a profile
    
    assert len(voter_names) == len(rankings), "Error: the number of voters doesn't match the number of rankings"
    return PointedProfile([Voter(vr[0], ranking = Ranking(vr[1])) for vr in zip(voter_names, rankings)], voter)

def display_tally(tally):
    # display a nice table for a tally of a profile
    tbl = PrettyTable()
    candidates = sorted(tally.keys())
    
    tbl.add_column('',candidates)
    for c in candidates:
        tbl.add_column(c,[tally[c1][c] if c1 != c else 0 for c1 in candidates])
    print tbl


In [8]:
#%run profile

prof = create_profile(["i", "j", "k"], [(A, B, C, D), (B, C, A, D), (C, A, B, D)])

pprof = create_pointed_profile(["i", "j", "k"], [(A, B, C, D), (B, C, A, D), (C, A, B, D)], "k")


print
prof.display()
print
print "all rankings associated with this profile {}".format(prof.all_rankings)
print "all rankings associated with this profile {}".format(pprof.all_rankings)
print
print "Profile as a dictionary"
print prof.as_dict()
print
print "Pointed profile\n\n"
pprof.display()
print
print "Voter is {}".format(pprof.voter)
print "Voter is {}".format(pprof.voter.name)
print 
print 
updated_pprofile = pprof.create_new_profile((D, C, B, A))
updated_pprofile.display()
print
print
print "The support for {} over {} is {}".format(A, B, prof.support(A, B))
print "The support for {} over {} is {}".format(A, C, prof.support(A, C))
print "The support for {} over {} is {}".format(A, D, prof.support(A, D))
print "The support for {} over {} is {}".format(B, A, prof.support(B, A))
print "The support for {} over {} is {}".format(B, C, prof.support(B, C))
print "The support for {} over {} is {}".format(B, D, prof.support(B, D))
print "The support for {} over {} is {}".format(D, A, prof.support(D, A))
print "The support for {} over {} is {}".format(D, B, prof.support(D, B))
print "\n---------------\n"
print "The tally for this profile is\n"
print "Get voter {}: {} (name is {}, rank is {})".format("i", prof.get_voter("i"), prof.get_voter("i").name, prof.get_voter("i").ranking)
display_tally(prof.tally())

print "Profile with candidates A and B removed: "
p_removed = prof.remove_candidates([A,B])
p_removed.display()
display_tally(p_removed.tally())

print "\n-----------\n"

print "The number that rank {} 1st is  {}".format(A, prof.num_rank(A, level=1))
print "The number that rank {} 1st is  {}".format(B, prof.num_rank(B, level=1))
print "The number that rank {} 1st is  {}".format(C, prof.num_rank(C, level=1))
print "The number that rank {} 1st is  {}".format(D, prof.num_rank(D, level=1))
print "\n"
print "The number that rank {} 2nd is  {}".format(A, prof.num_rank(A, level=2))
print "The number that rank {} 2nd is  {}".format(B, prof.num_rank(B, level=2))
print "The number that rank {} 2nd is  {}".format(C, prof.num_rank(C, level=2))
print "The number that rank {} 2nd is  {}".format(D, prof.num_rank(D, level=2))
print "\n"

print "The number that rank {} 3rd is  {}".format(A, prof.num_rank(A, level=3))
print "The number that rank {} 3rd is  {}".format(B, prof.num_rank(B, level=3))
print "The number that rank {} 3rd is  {}".format(C, prof.num_rank(C, level=3))
print "The number that rank {} 3rd is  {}".format(D, prof.num_rank(D, level=3))
print "\n"

print "The number that rank {} 4th is  {}".format(A, prof.num_rank(A, level=4))
print "The number that rank {} 4th is  {}".format(B, prof.num_rank(B, level=4))
print "The number that rank {} 4th is  {}".format(C, prof.num_rank(C, level=4))
print "The number that rank {} 4th is  {}".format(D, prof.num_rank(D, level=4))
print "\n"
print "Plurality tally is {}".format(prof.plurality_tally())

print "\n\n=================\n\n"

print "Create a new profile with a new ranking for the first voter (D, C, B, A)"
p2 = prof.create_new_profile(prof.voters[0], (D,C,B,A))

p2.display()

print "The number that rank {} 1st is  {}".format(A, p2.num_rank(A, level=1))
print "The number that rank {} 1st is  {}".format(B, p2.num_rank(B, level=1))
print "The number that rank {} 1st is  {}".format(C, p2.num_rank(C, level=1))
print "The number that rank {} 1st is  {}".format(D, p2.num_rank(D, level=1))
print "\n"
print "The number that rank {} 2nd is  {}".format(A, p2.num_rank(A, level=2))
print "The number that rank {} 2nd is  {}".format(B, p2.num_rank(B, level=2))
print "The number that rank {} 2nd is  {}".format(C, p2.num_rank(C, level=2))
print "The number that rank {} 2nd is  {}".format(D, p2.num_rank(D, level=2))
print "\n"

print "The number that rank {} 3rd is  {}".format(A, p2.num_rank(A, level=3))
print "The number that rank {} 3rd is  {}".format(B, p2.num_rank(B, level=3))
print "The number that rank {} 3rd is  {}".format(C, p2.num_rank(C, level=3))
print "The number that rank {} 3rd is  {}".format(D, p2.num_rank(D, level=3))
print "\n"

print "The number that rank {} 4th is  {}".format(A, p2.num_rank(A, level=4))
print "The number that rank {} 4th is  {}".format(B, p2.num_rank(B, level=4))
print "The number that rank {} 4th is  {}".format(C, p2.num_rank(C, level=4))
print "The number that rank {} 4th is  {}".format(D, p2.num_rank(D, level=4))
print "\n"

display_tally(p2.tally())
print "Plurality tally is {}".format(p2.plurality_tally())


print "\nTesting equality of profiles"
prof = create_profile(["i", "j", "k"],  [(A, B, C, D), (B, C, A, D), (C, A, B, D)])
prof2 = create_profile(["i", "j", "k"], [(A, B, C, D), (B, C, A, D), (C, A, B, D)])
prof3 = create_profile(["j", "i", "k"], [(A, B, C, D), (B, C, A, D), (C, A, B, D)])
prof4 = create_profile(["i", "j", "k"], [(B, A, C, D), (B, C, A, D), (C, A, B, D)])


print "Should be True: ", prof == prof2
print "Should be False: ", prof == prof3
print "Should be False: ", prof == prof4

print "Should be False: ", prof != prof2
print "Should be True: ", prof != prof3
print "Should be True: ", prof != prof4



+---+---+---+
| i | j | k |
+---+---+---+
| A | B | C |
| B | C | A |
| C | A | B |
| D | D | D |
+---+---+---+

all rankings associated with this profile [('A', 'C', 'B', 'D'), ('A', 'C', 'D', 'B'), ('A', 'B', 'C', 'D'), ('A', 'B', 'D', 'C'), ('A', 'D', 'C', 'B'), ('A', 'D', 'B', 'C'), ('C', 'A', 'B', 'D'), ('C', 'A', 'D', 'B'), ('C', 'B', 'A', 'D'), ('C', 'B', 'D', 'A'), ('C', 'D', 'A', 'B'), ('C', 'D', 'B', 'A'), ('B', 'A', 'C', 'D'), ('B', 'A', 'D', 'C'), ('B', 'C', 'A', 'D'), ('B', 'C', 'D', 'A'), ('B', 'D', 'A', 'C'), ('B', 'D', 'C', 'A'), ('D', 'A', 'C', 'B'), ('D', 'A', 'B', 'C'), ('D', 'C', 'A', 'B'), ('D', 'C', 'B', 'A'), ('D', 'B', 'A', 'C'), ('D', 'B', 'C', 'A')]
all rankings associated with this profile [('A', 'C', 'B', 'D'), ('A', 'C', 'D', 'B'), ('A', 'B', 'C', 'D'), ('A', 'B', 'D', 'C'), ('A', 'D', 'C', 'B'), ('A', 'D', 'B', 'C'), ('C', 'A', 'B', 'D'), ('C', 'A', 'D', 'B'), ('C', 'B', 'A', 'D'), ('C', 'B', 'D', 'A'), ('C', 'D', 'A', 'B'), ('C', 'D', 'B', 'A'), ('B', 'A

## Voting Methods

Voting methods are functions accepting a profile and returning a list of candidates (the list of candidates is always sorted). 

Currently, 11 voting methods are implemented: baldwin, borda, condorcet, coombs, copeland, hare, maxmin, plurality, plurality_with_runoff, nanson, nanson_weak. 

As a helper function, majority rule is also implemented. 

In [9]:
#%%writefile ./voting/voting_methods.py

from math import ceil
import numpy as np
import random

from voting.profile import generate_linear_orderings

# Voting methods

def plurality(profile):
    """Plurality"""

    plurality_tally = profile.plurality_tally()
    max_plurality_score = max(plurality_tally.values())
    
    return sorted([c for c in plurality_tally.keys() if plurality_tally[c] == max_plurality_score])

def majority(profile):
    '''Majority'''
    
    maj_size = profile.strict_maj_size()
    plurality_scores = profile.plurality_tally()
    maj_winner = [c for c in profile.candidates if c in plurality_scores.keys() and plurality_scores[c] >= maj_size]
    
    return sorted(maj_winner)

def borda(profile):
    """Borda"""
    
    candidates = profile.candidates
    borda_scores = profile.borda_scores()
    max_borda_score = max(borda_scores.values())
    
    return sorted([c for c in candidates if borda_scores[c] == max_borda_score])

def copeland(profile): 
    """Copeland"""
    
    tally  = profile.tally()
    candidates = tally.keys()
    copeland_scores = {c:len([1 for c2 in tally[c].keys() if tally[c][c2] > 0]) - len([1 for c2 in tally[c].keys() if tally[c][c2] < 0]) for c in candidates}
    max_copeland_score = max(copeland_scores.values())
    return sorted([c for c in candidates if copeland_scores[c] == max_copeland_score])


def maxmin(profile): 
    """MaxMin"""
    
    candidates = profile.candidates
    min_support = {c: min([profile.support(c,_c) for _c in candidates if _c != c]) for c in candidates}
    max_overall_support = max(min_support.values())
    return sorted([c for c in candidates if min_support[c] == max_overall_support])

def plurality_with_runoff(profile):
    """PluralityWRunoff"""

    plurality_tally = profile.plurality_tally()
    max_plurality_score = max(plurality_tally.values())
    first = [c for c in profile.candidates if plurality_tally[c] == max_plurality_score]
    if len(first) > 1:
        runoff_candidates = first
    else:
        second_plurality_score = list(reversed(sorted(plurality_tally.values())))[1]
        second = [c for c in profile.candidates if plurality_tally[c] == second_plurality_score]
        runoff_candidates = first + second
        
    runoff_profile = profile.remove_candidates([c for c in profile.candidates if c not in runoff_candidates])
    return plurality(runoff_profile)  

def hare(profile):
    """Hare"""
    
    candidates = profile.candidates
        
    plurality_scores = profile.plurality_tally()
    maj_winner = majority(profile)
    
    updated_profile = profile
    while len(maj_winner) == 0:
        min_plurality_score = min(plurality_scores.values())
        lowest_first_place_votes = [c for c in candidates if c in plurality_scores.keys() and plurality_scores[c] == min_plurality_score]
        
        # remove lowest plurality winners
        updated_profile = updated_profile.remove_candidates(lowest_first_place_votes)        
        if updated_profile.is_empty():
            maj_winner = sorted(lowest_first_place_votes)
        else:
            plurality_scores = updated_profile.plurality_tally()
            maj_winner = majority(updated_profile)
    return maj_winner

def coombs(profile):
    """Coombs"""
    
    maj_winner = majority(profile)
    
    updated_profile = profile
    while len(maj_winner) == 0:
        
        candidates, num_candidates = updated_profile.candidates, updated_profile.num_candidates
        last_place_scores = {c: updated_profile.num_rank(c,level=num_candidates) for c in candidates}
        max_last_place_score = max(last_place_scores.values())
        greatest_last_place_votes = [c for c in candidates if c in last_place_scores.keys() and last_place_scores[c] == max_last_place_score]
        
        # remove lowest plurality winners
        updated_profile = updated_profile.remove_candidates(greatest_last_place_votes)
        
        if updated_profile.is_empty():
            maj_winner = sorted(greatest_last_place_votes)
        else:
            maj_winner = majority(updated_profile)
    return maj_winner


def strict_nanson(profile):
    """StrictNanson"""
    
    borda_scores = profile.borda_scores()
    avg_borda_scores = np.mean(borda_scores.values())
    below_borda_avg_candidates = [c for c in profile.candidates if borda_scores[c] < avg_borda_scores]
    
    updated_profile = profile.remove_candidates(below_borda_avg_candidates)

    winners = updated_profile.candidates if updated_profile.num_candidates == 1 else []
    
    while len(winners) == 0: 
        
        borda_scores = updated_profile.borda_scores()
        avg_borda_scores = np.mean(borda_scores.values())
    
        below_borda_avg_candidates = [c for c in updated_profile.candidates if borda_scores[c] < avg_borda_scores]
        updated_profile = updated_profile.remove_candidates(below_borda_avg_candidates)
        
        if len(below_borda_avg_candidates) == 0:
            winners = sorted(updated_profile.candidates)
        elif updated_profile.num_candidates == 1:
            winners = updated_profile.candidates
        
    return winners 

def weak_nanson(profile):
    """WeakNanson"""
    
    borda_scores = profile.borda_scores()
    avg_borda_scores = np.mean(borda_scores.values())
    below_borda_avg_candidates = [c for c in profile.candidates if borda_scores[c] <= avg_borda_scores]
    
    updated_profile = profile.remove_candidates(below_borda_avg_candidates)

    winners = updated_profile.candidates if updated_profile.num_candidates == 1 else []
    
    if len(below_borda_avg_candidates) == profile.num_candidates: 
        winners = sorted(profile.candidates)
    else:
        while len(winners) == 0: 
        
            borda_scores = updated_profile.borda_scores()
            avg_borda_scores = np.mean(borda_scores.values())

            below_borda_avg_candidates = [c for c in updated_profile.candidates if borda_scores[c] <= avg_borda_scores]
        
            updated_profile = updated_profile.remove_candidates(below_borda_avg_candidates)

            if updated_profile.num_candidates  == 0:
                winners = sorted(below_borda_avg_candidates)
            elif updated_profile.num_candidates == 1:
                winners = updated_profile.candidates
        
    return winners 

def baldwin(profile):
    """Baldwin"""

    borda_scores = profile.borda_scores()
    candidates = profile.candidates
    min_borda_score = min(borda_scores.values())
    last_place_borda_scores = [c for c in candidates if c in borda_scores.keys() and borda_scores[c] == min_borda_score]
        
    # remove lowest plurality winners
    updated_profile = profile.remove_candidates(last_place_borda_scores)
    
    winners = list()
    if updated_profile.num_candidates == 0: 
        winners = sorted(last_place_borda_scores)
    
    while len(winners) == 0:
        
        borda_scores = updated_profile.borda_scores()
                
        candidates = updated_profile.candidates
        min_borda_score = min(borda_scores.values())
        last_place_borda_scores = [c for c in candidates if c in borda_scores.keys() and borda_scores[c] == min_borda_score]
           
        # remove lowest borda scores
        updated_profile = updated_profile.remove_candidates(last_place_borda_scores)
        
        if updated_profile.is_empty():
            winners = sorted(last_place_borda_scores)
        elif updated_profile.num_candidates == 1:
            winners = updated_profile.candidates  
    
    return winners 

def condorcet(profile):
    """Condorcet"""
    
    tally = profile.tally()
    cond_winner = filter(lambda c: all([tally[c][_] > 0 for _ in tally[c].keys()]), profile.candidates)
    return sorted(cond_winner) if len(cond_winner) > 0 else sorted(profile.candidates)



###
#Helper functions
###

def create_tie_breaker(lin_order):
    # given a linear ordering, create a tie breaker function
    
    return lambda cands: sorted(list(cands), key=lambda c: lin_order.index(c))[0]

def create_all_tie_breakers(candidate_names):
    
    # create all tie breakers for a set of candidates 
    # (each linear ordering gives a different tie-breaker)
    
    linear_orders = generate_linear_orderings(candidate_names)
    
    return [create_tie_breaker(lo) for lo in linear_orders]
    
def tie_breaker_cand_names(winners):
    
    # use the tie breaking that returns the candidate
    # whose name is alphabetically  earlies
    
    return sorted(winners)[0]

def tie_breaker_random(winners):
    
    # break time by  uniform probability over the winners
    return random.choice(winners)


def vms_to_str(voting_methods):
    # return a string of the voting method names
    return ','.join([_.__doc__ for _ in voting_methods])

def dom_to_str(dom):
    # return a string of the dominance tuple (weak_dom_name, strict_dom_name)
    return '|'.join(list(dom))

def str_to_dom(dom_str):
    
    return tuple(dom_str.split("|"))
    
def same_voting_methods_string(vms_str1, vms_str2):
    
    # test if the vothing methods strings are the same, e.g., 
    # the following should return true: "Borda", " Borda"; 
    # "Borda, Plurality", "Plurality, Borda"; and 
    # "Borda, Plurality, Plurality", "Plurality, Borda"
    vms1 = [vm_str.strip() for vm_str in vms_str1.split(",")]
    vms2 = [vm_str.strip() for vm_str in vms_str2.split(",")]
    
    return set(vms1) == set(vms2)



In [10]:

#print "Tie breaker for [A, B] is {}".format(tie_breaker_cand_names([A, B]))






same_voting_methods_string("Borda", "Borda")
same_voting_methods_string("Plurality", "Borda")
same_voting_methods_string("Borda, Plurality", "Borda, Plurality")
same_voting_methods_string("Borda", "  Borda")
same_voting_methods_string("Borda", "Plurality, Borda")
prof1 = create_profile(["1", "2", "3", "4"], [(A, B, C),
                                               (B, C, A), 
                                               (C, A, B),
                                               (C, B, A)])

prof2 = create_profile(["1", "2", "3", "4"], [(A, B, C),
                                               (A, C, B), 
                                               (B, A, C),
                                               (C, B, A)])

prof2 = create_profile(["1", "2", "3", "4", "5", "6"], [(A, B, C),
                                               (A, C, B), 
                                               (B, C, A),(B, C, A),
                                               (C, B, A),(C, B, A)])


prof3 = create_profile(["1", "2", "3", "4", "5"], [(C, B, A),
                                               (A, C, B), 
                                               (B, A, C),(C, B, A),
                                               (A, C, B)])

prof4 = create_profile(["1", "2", "3", "4"], [(A, B, C, D),
                                                (B, D, C, A),
                                                (C, A, B, D),
                                                (C, A, B, D)])


prof5 = create_profile(["1", "2", "3"], [(A, B, C, D),
                                                (B, A, C, D),
                                                (D, B, A, C)])


prof6 = create_profile(["1", "2", "3"], [(A, B, C, D),
                                                (B, D, C, A),
                                                (C, A, B, D)])

prof7 = create_profile(["1", "2", "3", "4", "5"], [(A, B, C),
                                               (A, B, C), 
                                               (B, A, C),(C, B, A),
                                               (C, B, A)])
manipulator = '1'
new_ordering1 = (B, A, C)

new_ordering2 = (A, C, B)

new_ordering3 = (B, A, C)
new_ordering4 = (B, D, C, A)
new_ordering5 = (A, C, D, B)
new_ordering6 = (A, D, B, C)
new_ordering7 = (A, C, B)

new_ordering = new_ordering6

the_prof = prof6


the_prof.display()
print "Plurality winners {}".format(plurality(the_prof))
print "Strict majority size is {}".format(the_prof.strict_maj_size())
print "Majority winners {}".format(majority(the_prof))
print "Borda winners {}".format(borda(the_prof))
print "Borda scores {}".format(the_prof.borda_scores())
print "Copeland winners {}".format(copeland(the_prof))
print "PluralityWRunoff winners {}".format(plurality_with_runoff(the_prof))
print "Hare winners {}".format(hare(the_prof))
print "Coombs winners {}".format(coombs(the_prof))
print "Nanson winners {}".format(strict_nanson(the_prof))
print "Nanson weak winners {}".format(weak_nanson(the_prof))
print "Baldwin winners {}".format(baldwin(the_prof))
print "Condorcet winners {}".format(condorcet(the_prof))
print "Maxmin winners {}".format(maxmin(the_prof))
print "\n\n=========\n\n"


updated_profile = the_prof.create_new_profile(the_prof.get_voter(manipulator), new_ordering)

updated_profile.display()
print "Updated profile  strict majority size is {}".format(updated_profile.strict_maj_size())
print "Updated profile majority winners {}".format(majority(updated_profile))
print "Updated profile Borda scores {}".format(updated_profile.borda_scores())
print
print "The maniuplator is {}".format(manipulator)
print "   True ordering {}".format(the_prof.get_voter(manipulator).ranking)
print "   New ordering {} ".format(updated_profile.get_voter(manipulator).ranking)
print 
print "Previous Borda winners {}".format(borda(the_prof))
print "New Borda winners {}".format(borda(updated_profile))
print 
#print "is sure manip: ", is_sure_manipulation([borda], "strong_dom", the_prof, the_prof.get_voter(manipulator), new_ordering1)
print 
print "Previous Plurality winners {}".format(plurality(the_prof))
print "New Plurality winners {}".format(plurality(updated_profile))
print 
print "Previous Copeland winners {}".format(copeland(the_prof))
print "New Copeland winners {}".format(copeland(updated_profile))
print 
print "Previous Maxmin winners {}".format(maxmin(the_prof))
print "New Maxmin winners {}".format(maxmin(updated_profile))
print 
print "Previous PluralityWRunoff winners {}".format(plurality_with_runoff(the_prof))
print "New PluralityWRunoff winners {}".format(plurality_with_runoff(updated_profile))
print 
print "Previous Hare winners {}".format(hare(the_prof))
print "New Hare winners {}".format(hare(updated_profile))
print 
print "Previous Coombs winners {}".format(coombs(the_prof))
print "New Coombs winners {}".format(coombs(updated_profile))
print 
print "Previous StrictNanson winners {}".format(strict_nanson(the_prof))
print "New StrictNanson winners {}".format(strict_nanson(updated_profile))
print 
print "Previous WeakNanson  winners {}".format(weak_nanson(the_prof))
print "New WeakNanson  winners {}".format(weak_nanson(updated_profile))
print 
print "Previous Baldwin winners {}".format(baldwin(the_prof))
print "New Baldwin winners {}".format(baldwin(updated_profile))
print 
print "Previous Condorcet winners {}".format(condorcet(the_prof))
print "New Condorcet winners {}".format(condorcet(updated_profile))
print "\n\n=========\n\n"


+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| A | B | C |
| B | D | A |
| C | C | B |
| D | A | D |
+---+---+---+
Plurality winners ['A', 'B', 'C']
Strict majority size is 2
Majority winners []
Borda winners ['B']
Borda scores {'A': 5, 'C': 5, 'B': 6, 'D': 2}
Copeland winners ['A', 'B', 'C']
PluralityWRunoff winners ['A', 'B', 'C']
Hare winners ['A', 'B', 'C']
Coombs winners ['A', 'B', 'C']
Nanson winners ['A', 'B', 'C']
Nanson weak winners ['A', 'B', 'C']
Baldwin winners ['A', 'B', 'C']
Condorcet winners ['A', 'B', 'C', 'D']
Maxmin winners ['A', 'B', 'C']




+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| A | B | C |
| D | D | A |
| B | C | B |
| C | A | D |
+---+---+---+
Updated profile  strict majority size is 2
Updated profile majority winners []
Updated profile Borda scores {'A': 5, 'C': 4, 'B': 5, 'D': 4}

The maniuplator is 1
   True ordering A B C D
   New ordering A D B C 

Previous Borda winners ['B']
New Borda winners ['A', 'B']


Previous Plurality winners ['A', 'B', 'C']
New Pl